In [1]:
from math import sqrt
from numpy import concatenate
import numpy as np
from matplotlib import pyplot
import os

import pandas as pd

from scipy.stats import uniform

from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error

from keras.models import Sequential, Model
from keras.layers import Activation, Input, Dense, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Flatten, GRU, LSTM
from keras.layers.advanced_activations import ELU
from keras.utils import np_utils
from keras.wrappers.scikit_learn import KerasRegressor
from keras.optimizers import *
from keras import backend as K
K.set_image_dim_ordering('th')

import tensorflow as tf
from pymlx import *

import warnings
warnings.filterwarnings('ignore')

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
# from IPython.display import Audio

Using TensorFlow backend.


In [2]:
tf.__version__

'1.5.0'

In [4]:
def make_windowed(dataset, seq_length, horizon, feat_cols, resp_cols, region_col, split=0.2, resp_width=0,
                        predict_current=False):
    """
    This function takes data and creates a windowed dataframe to be used in time-series analysis. 
    dataset: [panda df] the raw dataframe that a time series windowed df will be created from
    seq_length: [int] the sequence length, the amount of time to be used to perform the 
                time-series analysis
    horizon: [int] how far into the future you wish to predict
    feat_cols: [list] a list of column names that make up the feature space
    resp_cols: [list] a list of column names that make up the response
    region_col: [str] the name of the column that different time-series will be created on, i.e. 
                different regions that contain
                independent time-series.
    split: [float] A percent split for the test set, i.e. 0.2 equals a 80/20 split for the 
                train/test sets.
    resp_width: [int] If you want to predict out to a set distance you set the horizon to that 
                    time point and this value to 0, however if you want to predict every value 
                    between let's say now and some point in the future you set horizon to 1 and
                    the resp_width to that point. The algorithm will then predict every time point.
    predict_current: [bool] horizon needs to be set to 1 for this to work. This will predict at 
                    the current time so if there is a sequence length of 2 instead of forecasting 
                    out the horizon length, the model
                    will predict at the current time.
    """

    if ((predict_current) and (horizon is not 1)):
        raise ValueError('If predict_current is set to True, then Horizon must be set to 1.')
    else:
        pass

    if (any(dataset[feat_cols + resp_cols].isnull().sum()) != 0):
        raise ValueError('There is missing data in at least one of the columns supplied in keep_cols. \
                         Please impute this missing data as needed.')
    else:
        pass

    # check to see if there are same features in both the response and the features list
    resp_and_feats = [var for var in feat_cols if var in resp_cols]

    regions = list(dataset[region_col].unique())
    big_dict = {}

    for i in range(len(regions)):
        big_dict.update({regions[i]: dataset[dataset[region_col] == regions[i]]})
    features = len(feat_cols)
    response = len(resp_cols)
    if resp_width == 0:
        train_X_all, test_X_all = np.empty((0, seq_length, features)), np.empty((0, seq_length, features))
        train_y_all, test_y_all = np.empty((0, response)), np.empty((0, response))
    else:
        train_X_all, test_X_all = np.empty((0, sequence_length, features)), np.empty((0, sequence_length, features))
        train_y_all, test_y_all = np.empty((0, resp_width, response)), np.empty((0, resp_width, response))

    for region in regions:
        big_dict[region] = big_dict[region][resp_cols + feat_cols]
        if resp_and_feats:
            big_dict[region] = big_dict[region].loc[:, ~big_dict[region].columns.duplicated()]
        else:
            pass

        length = len(big_dict[region])
        test_length = int(length * split)  # 20% test set

        df_x = big_dict[region][feat_cols]
        df_y = big_dict[region][resp_cols]
        ts_x = df_x.values
        ts_y = df_y.values

        train_length = length - test_length

        train_X, train_y, test_X, test_y = [], [], [], []

        if (predict_current):
            z = 2
            q = 1
        else:
            z = 1
            q = 0

        if (resp_width != 0):
            for i in range(train_length - sequence_length - horizon - resp_width):
                train_X.append(ts_x[i:i + sequence_length])
                train_y.append(ts_y[i + sequence_length + horizon - z:i + sequence_length + horizon - z + resp_width])
            for i in range(-test_length, -resp_width):
                test_X.append(ts_x[i - sequence_length - horizon + 1:i - horizon + 1])
                test_y.append(ts_y[i - q:i - q + (resp_width)])

        else:
            for i in range(train_length - seq_length - horizon):
                train_X.append(ts_x[i:i + seq_length])
                train_y.append(ts_y[i + seq_length + horizon - z])
            for i in range(-test_length, 0):
                test_X.append(ts_x[i - seq_length - horizon + 1:i - horizon + 1])
                test_y.append(ts_y[i - q])

        train_X, train_y = np.array(train_X), np.array(train_y)
        test_X, test_y = np.array(test_X), np.array(test_y)

        train_X_all = np.append(train_X_all, train_X, axis=0)
        test_X_all = np.append(test_X_all, test_X, axis=0)
        train_y_all = np.append(train_y_all, train_y, axis=0)
        test_y_all = np.append(test_y_all, test_y, axis=0)

    # normalize
    mean_x = train_X_all.mean(0)
    std_x = train_X_all.std(0)

    train_X = (train_X_all - mean_x) / std_x
    test_X = (test_X_all - mean_x) / std_x
    train_X = train_X.astype('float32')
    test_X = test_X.astype('float32')

    mean_y = train_y_all.mean(0)
    std_y = train_y_all.std(0)

    train_y = (train_y_all - mean_y) / std_y
    test_y = (test_y_all - mean_y) / std_y
    train_y = train_y.astype('float32')
    test_y = test_y.astype('float32')

    if resp_width != 0:
        std_y = std_y.ravel()
        mean_y = mean_y.ravel()
        train_y = train_y.reshape(train_y.shape[0], train_y.shape[1] * response)
        test_y = test_y.reshape(test_y.shape[0], test_y.shape[1] * response)
    else:
        pass

    print('train_X shape:', np.shape(train_X))
    print('train_y shape:', np.shape(train_y))
    print('test_X shape:', np.shape(test_X))
    print('test_y shape:', np.shape(test_y))

    return train_X, test_X, train_y, test_y, mean_x, std_x, mean_y, std_y

In [5]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))


def compute_mape_minMax(model, x, y):
    forecasts = model.predict(x, batch_size=len(x))
    forecasts = (forecasts *  (max_soilT - min_soilT)) + min_soilT
    y_denom = (y * (max_soilT - min_soilT)) + min_soilT
    return np.mean(np.abs((y_denom - forecasts) / y_denom))


def compute_mape_centerScale(model, x, y):
    forecasts = model.predict(x)
    forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    sub_result = np.abs((y_denom - forecasts) / y_denom )
    # remove rows that are divided by 0 or a very small number
    idxinf = np.where(sub_result == np.inf)
    idx = np.where(sub_result > 1)
    total_remove = len(idxinf[0]) + len(idx[0])
    #print('Removed %f percent of the data, a total of %d rows.'% (((total_remove/len(forecasts))*100), total_remove))
    sub_result = np.delete(sub_result, idxinf[0],0)
    sub_result = np.delete(sub_result, idx[0], 0)
    result = (np.mean(sub_result))
    return result


def compute_mae(model, x, y):
    forecasts = model.predict(x)
#     forecasts = forecasts.reshape(forecasts.shape[0],dense)
    forecasts = (forecasts * std_y) + mean_y
    y_denom = (y * std_y) + mean_y
    result = mean_absolute_error(y_pred=forecasts, y_true=y_denom)
    return result


def test(model):
    # print('Train Mean Absolute Percentage Error (MAPE): {0:.3f}'.format(compute_mape_centerScale(model, train_X, train_y)))
    # print('Test Mean Absolute Percentage Error (MAPE): {0:.3f}'.format(compute_mape_centerScale(model, test_X, test_y)))
    return(compute_mape_centerScale(model, train_X, train_y), compute_mape_centerScale(model, test_X, test_y))

def test_mae(model):
    yhat_train = model.predict(train_X)
    yhat_test = model.predict(test_X)
    return(compute_mae(model, train_X, train_y), compute_mae(model, test_X, test_y))


def report(train, test):
    train_mean = np.asarray(train).mean()
    train_std = np.asarray(train).std()
    test_mean = np.asarray(test).mean()
    test_std = np.asarray(test).std()
    train = str('Training MAE %f ± %f'% (train_mean, train_std))
    test = str('Testing MAE %f ± %f'% (test_mean, test_std))
    return(train, test)

### Load Dataset

In [15]:
CWD = os.getcwd()
FILE_PATH = os.path.join(CWD, "../data/time_series_soilM_averaged.csv")
dataset = pd.read_csv(FILE_PATH)
dataset.head()
dataset['date'] = pd.to_datetime(dataset['date'])
dataset = dataset.set_index(['date'])
dataset = dataset.reset_index()

,date,field,soilM_avg,precip.cm,tair.C,rh.pct,wind_sp.m_per_s,irradiance.w_per_m.2,average_sand,average_silt,average_clay
0,2011-10-29,AES,0.212173,0.045,7.654167,68.083333,3.179167,121.708333,14.891866,60.796955,24.311179
1,2011-10-30,AES,0.210993,0.369,9.354167,78.250000,3.562500,72.500000,14.891866,60.796955,24.311179
2,2011-10-31,AES,0.209983,0.023,6.945833,68.625000,5.112500,122.666667,14.891866,60.796955,24.311179
3,2011-11-01,AES,0.209532,0.000,2.225000,66.250000,3.220833,127.500000,14.891866,60.796955,24.311179
4,2011-11-02,AES,0.207929,0.000,4.862500,44.958333,6.091667,124.041667,14.891866,60.796955,24.311179


In [14]:
dataset.columns

Index(['date', 'field', 'soilM_avg'], dtype='object')

In [22]:
dataset.isnull().sum()

date                      0
field                     0
soilM_avg               248
precip.cm                 0
tair.C                    0
rh.pct                    0
wind_sp.m_per_s           0
irradiance.w_per_m.2      0
average_sand              0
average_silt              0
average_clay              0
dtype: int64

### Set Parameters

In [20]:
resp_cols = ['soilM_avg']
feat_cols = ['precip.cm', 'tair.C', 'rh.pct', 'wind_sp.m_per_s', 'irradiance.w_per_m.2', 'average_clay',
       'average_sand', 'average_silt']
keep_cols = resp_cols + feat_cols

sequence_length = 10
horizon = 3
resp_width = 0
dense = len(resp_cols)

### Make Windowed DF

In [21]:
train_X, test_X, train_y, test_y, mean_x, std_x, mean_y, std_y = make_windowed(dataset=dataset, 
                                                                                     seq_length=sequence_length,
                                                                                     horizon=horizon,
                                                                                     resp_cols=resp_cols,
                                                                                     feat_cols=feat_cols,
                                                                                     region_col='field',
                                                                                     resp_width=resp_width,
                                                                                     predict_current=False)
dim = train_X.shape[2]

ValueError: There is missing data in at least one of the columns supplied in keep_cols.                          Please impute this missing data as needed.